In [2]:
from bs4 import BeautifulSoup as bs
import requests

#need to look at terms and conditions to know if you can scrape the website
#most websites have a file called robots.txt that you can look at to see
#if scraping is allowed. ex: google.com/robots.txt
url = 'http://quotes.toscrape.com/'

In [3]:
#get page with requests
response = requests.get(url)
response

<Response [200]>

In [5]:
soup = bs(response.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Quotes to Scrape
  </title>
  <link href="/static/bootstrap.min.css" rel="stylesheet"/>
  <link href="/static/main.css" rel="stylesheet"/>
 </head>
 <body>
  <div class="container">
   <div class="row header-box">
    <div class="col-md-8">
     <h1>
      <a href="/" style="text-decoration: none">
       Quotes to Scrape
      </a>
     </h1>
    </div>
    <div class="col-md-4">
     <p>
      <a href="/login">
       Login
      </a>
     </p>
    </div>
   </div>
   <div class="row">
    <div class="col-md-8">
     <div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
      <span class="text" itemprop="text">
       “The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
      </span>
      <span>
       by
       <small class="author" itemprop="author">
        Albert Einstein
       </small>
       <a href="/author/Albert

In [6]:
print(soup.title)

<title>Quotes to Scrape</title>


In [19]:
all_divs = soup.find_all('div',class_='quote')
all_divs[1]

<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“It is our choices, Harry, that show what we truly are, far more than our abilities.”</span>
<span>by <small class="author" itemprop="author">J.K. Rowling</small>
<a href="/author/J-K-Rowling">(about)</a>
</span>
<div class="tags">
            Tags:
            <meta class="keywords" content="abilities,choices" itemprop="keywords"/>
<a class="tag" href="/tag/abilities/page/1/">abilities</a>
<a class="tag" href="/tag/choices/page/1/">choices</a>
</div>
</div>

In [17]:
import pandas as pd
page_one_results = []
for div in all_divs:
    try:
        quote_text = div.find('span',class_='text').text
        author = div.find('small', class_='author').text
        quote_dict = {'quote':quote_text,
                     'author':author}
        page_one_results.append(quote_dict)
    except AttributeError as e:
        print(e)
page_one_df = pd.DataFrame(page_one_results)
page_one_df

,quote,author
0,“The world as we have created it is a process ...,Albert Einstein
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling
2,“There are only two ways to live your life. On...,Albert Einstein
3,"“The person, be it gentleman or lady, who has ...",Jane Austen
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe
5,“Try not to become a man of success. Rather be...,Albert Einstein
6,“It is better to be hated for what you are tha...,André Gide
7,"“I have not failed. I've just found 10,000 way...",Thomas A. Edison
8,“A woman is like a tea bag; you never know how...,Eleanor Roosevelt
9,"“A day without sunshine is like, you know, nig...",Steve Martin


In [22]:
all_quotes = []

for i in range(1,11):
    url = f'https://quotes.toscrape.com/page/{i}'
    response = requests.get(url)
    soup = bs(response.text, 'html.parser')
    all_divs = soup.find_all('div',class_='quote')
    
    for div in all_divs:
        try:
            quote_text = div.find('span',class_='text').text
            author = div.find('small', class_='author').text
            tags = div.find('div',class_='tags').find_all('a')
            tags_list=[]
            for tag in tags:
                tags_list.append(tag.text)
            quote_dict = {'quote':quote_text,
                         'author':author,
                         'tags':tags_list}
            
            
            all_quotes.append(quote_dict)
        except AttributeError as e:
            print(e)
            
df = pd.DataFrame(all_quotes)
df

,quote,author,tags
0,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]"
2,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]"
...,...,...,...
95,“Try not to become a man of success. Rather be...,Albert Einstein,"[adulthood, success, value]"
96,“It is better to be hated for what you are tha...,André Gide,"[life, love]"
97,"“I have not failed. I've just found 10,000 way...",Thomas A. Edison,"[edison, failure, inspirational, paraphrased]"
98,“A woman is like a tea bag; you never know how...,Eleanor Roosevelt,[misattributed-eleanor-roosevelt]


# chrome driver 

In [35]:
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

executable_path = {'executable_path':ChromeDriverManager().install()}
browser = Browser('chrome',**executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\alm0708\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [38]:
all_quotes = []
url = 'http://quotes.toscrape.com/'
browser.visit(url)

for i in range(1,5):    
    soup = bs(browser.html, 'html.parser')
    all_divs = soup.find_all('div',class_='quote')
    for div in all_divs:
        try:
            quote_text = div.find('span',class_='text').text
            author = div.find('small', class_='author').text
            tags = div.find('div',class_='tags').find_all('a')
            tags_list=[]
            for tag in tags:
                tags_list.append(tag.text)
                
            quote_dict = {'quote':quote_text,
                         'author':author,
                         'tags':tags_list}
            #print(quote_dict)
            
            all_quotes.append(quote_dict)
            
        except AttributeError as e:
            print(e)
    browser.links.find_by_partial_text('Next').click()
df = pd.DataFrame(all_quotes)
df

,quote,author,tags
0,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]"
2,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]"
5,“Try not to become a man of success. Rather be...,Albert Einstein,"[adulthood, success, value]"
6,“It is better to be hated for what you are tha...,André Gide,"[life, love]"
7,"“I have not failed. I've just found 10,000 way...",Thomas A. Edison,"[edison, failure, inspirational, paraphrased]"
8,“A woman is like a tea bag; you never know how...,Eleanor Roosevelt,[misattributed-eleanor-roosevelt]
9,"“A day without sunshine is like, you know, nig...",Steve Martin,"[humor, obvious, simile]"
